In [1]:
from multiprocessing import RawValue, RawArray, Semaphore, Lock, Process
import numpy as np
import ctypes
import numpy as np
import time

In [25]:
class SharedCounter(object):
    def __init__(self, initval=0):
        self.val = RawValue('i', initval)
        self.last_step_update_target = RawValue('i', initval)
        self.lock = Lock()

    def value(self):
        return self.val.value

class Barrier_v2:
    def __init__(self, n):
        self.n = n
        self.counter = SharedCounter(0)
        self.barrier = Semaphore(0)

    def wait(self,name):
        with self.counter.lock:
            self.counter.val.value += 1
            if self.counter.val.value % self.n == 0: 
                for i in range(self.n-1):
                    self.barrier.release()
                return
        
        print('block ' + name)
        self.barrier.acquire()
        print('release ' + name)

class Barrier:
    def __init__(self, n):
        self.n = n
        self.counter = SharedCounter(0)
        self.barrier = Semaphore(0)

    def wait(self,name):
        with self.counter.lock:
            self.counter.val.value += 1
            if self.counter.val.value == self.n:
                self.barrier.release()
                
        print('block ' + name)
        self.barrier.acquire()
        print('release ' + name)
        self.barrier.release()

In [23]:
class test(Process):
    def __init__(self,b,name):
        super(test, self).__init__()
        self.barrier = b
        self.name = name
        
    def run(self):
        print('activating ' + self.name)
        self.barrier.wait(self.name)
        print('terminating ' + self.name)
        print(self.barrier.barrier)
        
        time.sleep(10)
        
        print('Start ' + self.name)
        self.barrier.wait('X' + self.name)
        print('End ' + self.name)
        print(self.barrier.barrier)
        

In [20]:
b = Barrier(5)

processes = []
for i in range(5):
    processes.append(test(b,str(i)))
    time.sleep(1)
    processes[-1].start()
    
for t in processes:
    t.join()

activating 0
block 0
activating 1
block 1
activating 2
block 2
activating 3
block 3
activating 4
block 4
release 0
terminating 0
release 1
release 2
<Semaphore(value=0)>
terminating 1
release 3
terminating 2
<Semaphore(value=0)>
<Semaphore(value=0)>
terminating 3
release 4
<Semaphore(value=0)>
terminating 4
<Semaphore(value=1)>
Start 0
block X0
Start 1
Start 3
release X0
Start 2
block X1
block X3
Start 4
block X2
release X3
block X4
End 0
release X1
End 3
<Semaphore(value=0)>
<Semaphore(value=0)>
End 1
release X2
<Semaphore(value=0)>
End 2
release X4
End 4
<Semaphore(value=1)>
<Semaphore(value=1)>


In [26]:
b = Barrier_v2(5)

processes = []
for i in range(5):
    processes.append(test(b,str(i)))
    time.sleep(1)
    processes[-1].start()
    
for t in processes:
    t.join()

activating 0
block 0
activating 1
block 1
activating 2
block 2
activating 3
block 3
activating 4
release 0
release 2
terminating 4
release 3
release 1
terminating 0
terminating 2
terminating 3
<Semaphore(value=0)>
terminating 1
<Semaphore(value=0)>
<Semaphore(value=0)>
<Semaphore(value=0)>
<Semaphore(value=0)>
Start 2
Start 4
Start 1
Start 3
block X3
block X4
block X2
block X1
Start 0
release X4
End 0
release X1
release X2
release X3
End 4
<Semaphore(value=0)>
End 2
<Semaphore(value=0)>
End 1
End 3
<Semaphore(value=0)>
<Semaphore(value=0)>
<Semaphore(value=0)>
